In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

In [3]:
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_csv('../data/ml-100k/u.data', sep='\t', names=columns)
print("Dữ liệu gốc:")
print(df.head())

print("\nThông tin bộ dữ liệu:")
n_users = df['user_id'].nunique()
n_movies = df['movie_id'].nunique()
print(f"Số lượng người dùng: {n_users}")
print(f"Số lượng phim: {n_movies}") 
print(f"Tổng số đánh giá: {len(df)}")

Dữ liệu gốc:
   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596

Thông tin bộ dữ liệu:
Số lượng người dùng: 943
Số lượng phim: 1682
Tổng số đánh giá: 100000


In [4]:
# Test với dữ liệu nhỏ
data = {
    'Alice': [5, 3, 4, 4, np.nan],
    'User1': [3, 1, 2, 3, 3],
    'User2': [4, 3, 4, 3, 5],
    'User3': [3, 3, 1, 5, 4],
    'User4': [1, 5, 5, 2, 1]
}
item_names = [f'Item{i + 1}' for i in range(len(data['Alice']))]
example_df = pd.DataFrame(data, index=item_names).T
print("\nDữ liệu ví dụ:")
print(example_df)


Dữ liệu ví dụ:
       Item1  Item2  Item3  Item4  Item5
Alice    5.0    3.0    4.0    4.0    NaN
User1    3.0    1.0    2.0    3.0    3.0
User2    4.0    3.0    4.0    3.0    5.0
User3    3.0    3.0    1.0    5.0    4.0
User4    1.0    5.0    5.0    2.0    1.0


In [6]:
# Sử dụng pearsonr để tính tương quan giữa các cặp người dùng
def pearson_similarity(user1_ratings, user2_ratings):
  print("Rating của User1:", user1_ratings)
  print("Rating của User2:", user2_ratings)
  common_items_mask = user1_ratings.notna() & user2_ratings.notna()
  
  # Kiểm tra nếu có ít nhất một mục chung
  if common_items_mask.sum() == 0:
    return 0
  
  # Lấy các đánh giá chung
  common_ratings1 = user1_ratings[common_items_mask]
  common_ratings2 = user2_ratings[common_items_mask]
  
  # Pearson cần ít nhất 2 điểm dữ liệu
  if len(common_ratings1) < 2:
    return 0
  
  corr, _ = pearsonr(common_ratings1, common_ratings2)
  return corr

similarities = {}
alice_ratings = example_df.loc['Alice']
print("Đánh giá của Alice:", alice_ratings)
for user in example_df.index:
  if user != 'Alice':
    user_ratings = example_df.loc[user]
    similarity = pearson_similarity(alice_ratings, user_ratings)
    similarities[user] = similarity
print("Độ tương đồng của các user với Alice:")
print(similarities)
sorted_similarities = sorted(similarities.items(), key=lambda x : x[1], reverse=True)

for user, sim in sorted_similarities:
  print(f"User: {user}, Similarity: {sim:.4f}")

Đánh giá của Alice: Item1    5.0
Item2    3.0
Item3    4.0
Item4    4.0
Item5    NaN
Name: Alice, dtype: float64
Rating của User1: Item1    5.0
Item2    3.0
Item3    4.0
Item4    4.0
Item5    NaN
Name: Alice, dtype: float64
Rating của User2: Item1    3.0
Item2    1.0
Item3    2.0
Item4    3.0
Item5    3.0
Name: User1, dtype: float64
Rating của User1: Item1    5.0
Item2    3.0
Item3    4.0
Item4    4.0
Item5    NaN
Name: Alice, dtype: float64
Rating của User2: Item1    4.0
Item2    3.0
Item3    4.0
Item4    3.0
Item5    5.0
Name: User2, dtype: float64
Rating của User1: Item1    5.0
Item2    3.0
Item3    4.0
Item4    4.0
Item5    NaN
Name: Alice, dtype: float64
Rating của User2: Item1    3.0
Item2    3.0
Item3    1.0
Item4    5.0
Item5    4.0
Name: User3, dtype: float64
Rating của User1: Item1    5.0
Item2    3.0
Item3    4.0
Item4    4.0
Item5    NaN
Name: Alice, dtype: float64
Rating của User2: Item1    1.0
Item2    5.0
Item3    5.0
Item4    2.0
Item5    1.0
Name: User4, dtype: float64

In [8]:
K = 2 # Số lượng người dùng tương tự nhất
target_item = 'Item5'

# Lấy K người dùng tương tự nhất
neighbors = {user: sim for user, sim in sorted_similarities[:K] if sim > 0}

top_k_neighbors = sorted(neighbors.items(), key=lambda item: item[1], reverse=True)[:K]
print(f"\nTop {K} người dùng tương tự nhất với Alice:")
for user, sim in top_k_neighbors:
    print(f"User: {user}, Similarity: {sim:.4f}")



Top 2 người dùng tương tự nhất với Alice:
User: User1, Similarity: 0.8528
User: User2, Similarity: 0.7071


In [9]:
numerator = 0
denominator = 0

for user, sim in top_k_neighbors:
  neighbors_ratings = example_df.loc[user, target_item]
  numerator += sim * neighbors_ratings
  denominator += abs(sim)
  
predicted_rating = numerator / denominator if denominator != 0 else 0
print(f"Rating dự đoán của Alice cho {target_item} là: {predicted_rating:.4f}")

Rating dự đoán của Alice cho Item5 là: 3.9066


In [11]:
user_item_matrix = pd.pivot_table(df, index='user_id', columns='movie_id', values='rating')

print("\nMa trận người dùng - mục:")
print(user_item_matrix.head())
print("Kích thước ma trận:", user_item_matrix.shape)


Ma trận người dùng - mục:
movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2          4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5          4.0   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
1          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
5      

In [12]:
sparsity = float(user_item_matrix.isna().sum().sum()) / (user_item_matrix.shape[0] * user_item_matrix.shape[1])
print(f"\nTỷ lệ thưa thớt của ma trận người dùng - mục: {sparsity:.4f}")


Tỷ lệ thưa thớt của ma trận người dùng - mục: 0.9370


In [15]:
user_item_matrix_mean_centered = user_item_matrix.apply(lambda row : row - row.mean(), axis=1)

user_item_matrix_mean_centered.fillna(0, inplace=True)
# print("\nMa trận người dùng - mục đã trung bình hóa:")
# print(user_item_matrix_mean_centered.head())
# print("Kích thước ma trận:", user_item_matrix_mean_centered.shape)

user_similarity_matrix = pd.DataFrame(cosine_similarity(user_item_matrix_mean_centered), 
                                      index=user_item_matrix.index,
                                      columns=user_item_matrix.index)
print("Ma trận tương đồng User-User (5x5):")
display(user_similarity_matrix.iloc[:5, :5])

Ma trận tương đồng User-User (5x5):


user_id,1,2,3,4,5
user_id,,,,,
1,1.000000,0.043411,0.011051,0.059303,0.134514
2,0.043411,1.000000,0.013658,-0.017016,0.035770
3,0.011051,0.013658,1.000000,-0.059638,0.016037
4,0.059303,-0.017016,-0.059638,1.000000,0.007373
5,0.134514,0.035770,0.016037,0.007373,1.000000


In [17]:
def predict_user_based(user_id, item_id, user_item_matrix, similarity_matrix, k=5):
  user_sims = similarity_matrix.loc[user_id].drop(user_id)
  
  item_ratings = user_item_matrix[item_id]
  
  valid_neighbors_mask = (item_ratings.notna()) & (user_sims > 0)
  
  if not valid_neighbors_mask.any():
    user_mean = user_item_matrix.loc[user_id].mean()
    return user_mean if not np.isnan(user_mean) else 0
  
  neighbor_sims = user_sims[valid_neighbors_mask]
  neighbor_ratings = item_ratings[valid_neighbors_mask]
  
  if len(neighbor_sims) > k:
    top_k_indices = neighbor_sims.nlargest(k).index
    neighbor_sims = neighbor_sims[top_k_indices]
    neighbor_ratings = neighbor_ratings[top_k_indices]
  numerator = (neighbor_sims * neighbor_ratings).sum()
  denominator = neighbor_sims.sum()
  
  if denominator == 0:
    user_mean = user_item_matrix.loc[user_id].mean()
    return user_mean if not np.isnan(user_mean) else 0
  return numerator / denominator 

predicted = predict_user_based(1, 50, user_item_matrix, user_similarity_matrix, k=5)

actual = user_item_matrix.loc[1, 50]
print(f"\nRating thực tế của User 1 cho Item 50: {actual}")
print(f"Rating dự đoán của User 1 cho Item 50: {predicted:.4f}")


Rating thực tế của User 1 cho Item 50: 5.0
Rating dự đoán của User 1 cho Item 50: 5.0000


In [19]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_user_item_matrix = pd.pivot_table(train_df, index='user_id', columns='movie_id', values='rating')
train_mean_centered = train_user_item_matrix.apply(lambda row: row - row.mean(), axis=1).fillna(0)

train_user_similarity_matrix = pd.DataFrame(cosine_similarity(train_mean_centered),
                                            index=train_mean_centered.index,
                                            columns=train_mean_centered.index)

predictions = []
true_ratings = []

for _, row in test_df.iterrows():
  user_id = row['user_id']
  item_id = row['movie_id']
  actual_rating = row['rating']
  if user_id in train_user_similarity_matrix.index and item_id in train_user_item_matrix.columns:
        pred = predict_user_based(user_id, item_id, train_user_item_matrix, train_user_similarity_matrix, k=25)
        predictions.append(pred)
        true_ratings.append(actual_rating)

# Tính toán RMSE
rmse = np.sqrt(mean_squared_error(true_ratings, predictions))
print(f"RMSE trên tập test: {rmse:.4f}")

RMSE trên tập test: 1.0018
